In [1]:
    import cv2
    import pandas as pd
    from ultralytics import YOLO
    import numpy as np
    import pytesseract
    from datetime import datetime
    import re  # For regular expressions
    
    pytesseract.pytesseract.tesseract_cmd = "/opt/local/bin/tesseract"
    
    model = YOLO('/Users/macbookpro/Downloads/Licence-Plate-Detection-using-YOLO-V8-main/best.pt')
    
    def RGB(event, x, y, flags, param):
        if event == cv2.EVENT_MOUSEMOVE:
            point = [x, y]
            print(point)
    
    cv2.namedWindow('RGB')
    cv2.setMouseCallback('RGB', RGB)
    
    cap = cv2.VideoCapture('/Users/macbookpro/Downloads/carnumberplate-main 2/mycarplate.mp4')
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_time = int(1000 / fps)  # Time per frame in milliseconds
    
    my_file = open("/Users/macbookpro/Desktop/coco.txt", "r")
    data = my_file.read()
    class_list = data.split("\n") 
    
    area = [(27, 417), (16, 456), (1015, 451), (992, 417)]
    
    count = 0
    list1 = []
    processed_numbers = set()
    
    # Define state mappings
    states = {
        "AN": "Andaman and Nicobar", "AP": "Andhra Pradesh", "AR": "Arunachal Pradesh",
        "AS": "Assam", "BR": "Bihar", "CH": "Chandigarh", "DN": "Dadra and Nagar Haveli",
        "DD": "Daman and Diu", "DL": "Delhi", "GA": "Goa", "GJ": "Gujarat", "HR": "Haryana",
        "HP": "Himachal Pradesh", "JK": "Jammu and Kashmir", "KA": "Karnataka", "KL": "Kerala",
        "LD": "Lakshadweep", "MP": "Madhya Pradesh", "MH": "Maharashtra", "MN": "Manipur",
        "ML": "Meghalaya", "MZ": "Mizoram", "NL": "Nagaland", "OD": "Odissa", "PY": "Pondicherry",
        "PN": "Punjab", "RJ": "Rajasthan", "SK": "Sikkim", "TN": "Tamil Nadu", "TR": "Tripura",
        "UP": "Uttar Pradesh", "WB": "West Bengal", "CG": "Chhattisgarh", "TS": "Telangana",
        "JH": "Jharkhand", "UK": "Uttarakhand"
    }
    
    def clean_number_plate(text):
        text = text.strip()
        text = re.sub(r'[^A-Z0-9]', '', text)  # Remove non-alphanumeric characters
        return text
    
    def get_state_from_plate(plate):
        if plate and len(plate) >= 2:
            prefix = plate[:2]
            return states.get(prefix.upper(), "Unknown")
        return "Unknown"
    
    # Initialize the DataFrame
    car_plate_data = pd.DataFrame(columns=["NumberPlate", "Date", "Time", "State"])
    
    def expand_bbox(x1, y1, x2, y2, scale=0.2):
        """Expand the bounding box by a given scale factor."""
        width = x2 - x1
        height = y2 - y1
        x1_expanded = int(x1 - width * scale)
        y1_expanded = int(y1 - height * scale)
        x2_expanded = int(x2 + width * scale)
        y2_expanded = int(y2 + height * scale)
        return x1_expanded, y1_expanded, x2_expanded, y2_expanded
    
    while True:    
        ret, frame = cap.read()
        if not ret:
            break
    
        count += 1
        if count % 3 != 0:
            continue
    
        frame = cv2.resize(frame, (1020, 500))
        results = model.predict(frame)
        a = results[0].boxes.data
        px = pd.DataFrame(a).astype("float")
    
        new_data = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            
            d = int(row[5])
            c = class_list[d]
            cx = int(x1 + x2) // 2
            cy = int(y1 + y2) // 2
            result = cv2.pointPolygonTest(np.array(area, np.int32), ((cx, cy)), False)
            if result >= 0:
               # Expand the bounding box
               x1_expanded, y1_expanded, x2_expanded, y2_expanded = expand_bbox(x1, y1, x2, y2)
               crop = frame[y1_expanded:y2_expanded, x1_expanded:x2_expanded]
               gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
               gray = cv2.bilateralFilter(gray, 10, 20, 20)
    
               text = pytesseract.image_to_string(gray).strip()
               text = text.replace('(', '').replace(')', '').replace(',', '').replace(']','')
               text = clean_number_plate(text)
               if text and text not in processed_numbers:
                  processed_numbers.add(text) 
                  list1.append(text)
                  current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                  state = get_state_from_plate(text)
                  new_data.append({"NumberPlate": text, "Date": current_datetime.split()[0], "Time": current_datetime.split()[1], "State": state})
                  # Draw the expanded bounding box
                  cv2.rectangle(frame, (x1_expanded, y1_expanded), (x2_expanded, y2_expanded), (0, 255, 0), 2)
                  cv2.imshow('crop', crop)
    
        if new_data:
            new_df = pd.DataFrame(new_data)
            car_plate_data = pd.concat([car_plate_data, new_df], ignore_index=True)
        
        cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 0), 2)
        cv2.imshow("RGB", frame)
        if cv2.waitKey(frame_time) & 0xFF == 27:
            break
    
    cap.release()    
    cv2.destroyAllWindows()
    
    # Save the DataFrame to an Excel file
    car_plate_data.to_excel("carsfinals.xlsx", index=False)
    print("License plate data has been saved to 'car_plate_data.to_excel'")



0: 320x640 (no detections), 57.4ms
Speed: 2.1ms preprocess, 57.4ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 57.7ms
Speed: 2.0ms preprocess, 57.7ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 56.0ms
Speed: 1.5ms preprocess, 56.0ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 53.7ms
Speed: 1.6ms preprocess, 53.7ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 54.2ms
Speed: 1.5ms preprocess, 54.2ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 63.7ms
Speed: 2.4ms preprocess, 63.7ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)



2024-10-04 09:41:32.200 Python[22490:3253031] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-04 09:41:32.200 Python[22490:3253031] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


0: 320x640 1 LicensePlate, 55.3ms
Speed: 1.5ms preprocess, 55.3ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 LicensePlates, 53.5ms
Speed: 1.5ms preprocess, 53.5ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 53.5ms
Speed: 1.6ms preprocess, 53.5ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 LicensePlate, 54.8ms
Speed: 1.5ms preprocess, 54.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 53.2ms
Speed: 1.4ms preprocess, 53.2ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 55.2ms
Speed: 1.9ms preprocess, 55.2ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 53.8ms
Speed: 1.5ms preprocess, 53.8ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 53.5ms
Speed: 1.5ms preprocess, 53.5ms infer

ModuleNotFoundError: No module named 'openpyxl'